In [5]:
from pathlib import Path
import sys
import yaml
import time

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

DIR = Path('..')
sys.path.append(str(DIR))
from SCanalyzer import SCanalyzer
from SCanalyzer.busSim import BusSim, Config
from SCanalyzer.busSim.manager import managerFactory

%matplotlib inline

In [2]:
def get_yelp_api_key():
    with open("../config.yml", 'r') as yml:
        cfg = yaml.safe_load(yml)
        return cfg["yelp"]["api_key"]

def plot_background():
    background_path = "../data/plot/background/"
    city = gpd.read_file(background_path + "madison-meter-shp")
    lakes = gpd.read_file(background_path + "water-meter-shp")
    street = gpd.read_file(background_path + "street-meter-shp")
    # city = gpd.read_file(background_path + "madison-shp")
    # lakes = gpd.read_file(background_path + "water-shp")
    # street = gpd.read_file(background_path + "street-shp")

    ax = city.plot(color="lightgray", alpha=.2, figsize=(12, 12), zorder=2)
    lakes.plot(color="lightblue", ax=ax, zorder=1, alpha=0.8)
    street.plot(color="darkgray", alpha = .5, ax=ax, zorder=3)
    return ax

# route remove study

In [3]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
census_gdf = gtfo.load_census()
services_gdf = gtfo.load_yelp(api_key=get_yelp_api_key())

routes = [ 2,  4,  5,  6,  7,  8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22,
       23, 26, 27, 30, 31, 32, 33, 36, 39, 40, 44, 48, 49, 50, 51, 52, 55,
       67, 70, 72, 73, 75, 59, 63, 68, 78, 80, 84, 81, 82]
for route in routes:
    print(route)
    gtfo.set_batch_label(f"{route}-rm")
    config = Config(day="monday", elapse_time="00:30:00", interval="00:10:00", max_walking_min=10, route_remove=[route])
    config.set_starts(centroids=census_gdf)

    perf_df = pd.DataFrame(
            columns=["geometry", "start_time", "search_time", "add_service_time", "add_census_time"])
    result_gdf = gtfo.search(config, perf_df)
    gtfo.add_service_metrics(result_gdf, services_gdf, perf_df)
    gtfo.add_demographic_metrics(result_gdf, census_gdf, perf_df)
    result_gdf.to_csv(f"../out/result/result{route}.csv", index=False)
    perf_df.to_csv(f"../out/perf/perf{route}.csv", index=False)

NameError: name 'Gtfo' is not defined

In [ ]:
perf = pd.read_csv("../out/perf/perf80.csv")
perf["search_time"].sum(), perf["add_service_time"].sum(), perf["add_census_time"].sum(),

In [ ]:
sc = SCanalyzer("../data/mmt_gtfs.zip")
census_gdf = sc.load_census()
config = Config(day="monday", elapse_time="00:30:00", interval="10:00:00", max_walking_min=10)
# config.set_starts(points=[(43.073691, -89.387407), (43.073691, -89.387407)])
config.set_starts(centroids=census_gdf)
t0 = time.time()
result_gdf = sc.search(config)
t1 = time.time()
print(t1-t0)

services_gdf = sc.load_yelp(api_key=get_yelp_api_key())
sc.add_service_metrics(result_gdf, services_gdf)
t2 = time.time()
print(t2-t1)

sc.add_demographic_metrics(result_gdf, census_gdf) # SLOW
t3 = time.time()
print(t3-t2)
result_gdf["banks"].sum() #7263.0

14.490088939666748
6.0011491775512695
3.6119019985198975


4433.0

# Visualize simulation

In [ ]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
ax = plot_background()
gtfo.set_batch_label(f"80-rm")
example_gdf = gtfo.load_result_map(map_identifier="search-result-0-600!10")
example_gdf.plot(color="#ffbfba", ax=ax)

# AWS test

In [6]:
from SCanalyzer.busSim.manager import managerFactory
aws = managerFactory.create("aws", gtfs_path=None, out_path=None, borders=None)

In [ ]:
aws._create_repo()

{'repository': {'repositoryArn': 'arn:aws:ecr:ap-northeast-1:347664766527:repository/scanalyzer-lambda-handler', 'registryId': '347664766527', 'repositoryName': 'scanalyzer-lambda-handler', 'repositoryUri': '347664766527.dkr.ecr.ap-northeast-1.amazonaws.com/scanalyzer-lambda-handler', 'createdAt': datetime.datetime(2021, 6, 22, 22, 27, 10, tzinfo=tzlocal()), 'imageTagMutability': 'MUTABLE', 'imageScanningConfiguration': {'scanOnPush': False}, 'encryptionConfiguration': {'encryptionType': 'AES256'}}, 'ResponseMetadata': {'RequestId': '4416b639-eab1-4617-8ed5-136b907246d4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4416b639-eab1-4617-8ed5-136b907246d4', 'date': 'Tue, 22 Jun 2021 14:27:09 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '438'}, 'RetryAttempts': 0}}


In [ ]:
aws._ecr.get_authorization_token()['authorizationData'][0]

{'authorizationToken': 'QVdTOmV5SndZWGxzYjJGa0lqb2lSV1l2ZEhKV1ZGTnNkbFUxZDI5TFpXSktabk4yUmpGNFZXc3llRUppVVROYVoyVktWRzFxWnk5blIwTlRjM1ZPUjJST00ydG9PSEZwZEZnMVNUWjNkR3hpTmtkUE1YVnhibEJ5UkdoT1QyeEdTa2hJVmxWbWNuTTNZMlV5TDJrdk1rTkpVVGhHY1hWMGNqTnJXR05VZUZoSVoySnNka1JwUW1JeldHTnJWek4wYUVkdGFVOHZWa3RPWlcxU2RtVjNaRXBYZVVWWFRtZGxSbTV2YjBwSVVVbFdlbGN2UWpOcGJIVTBTVlY1UmtReVQwOHZlbXR1ZGtwSVYxVjZNRzVOTHpKbGFEbHpVQ3RhWkRBNVpIVmpRalZ6YjNrMVNtVklUalpCTTNKemFua3ZWRGhZUlRaMGNGZFhVbHAxWW1aUVRVNDVjM0pxWkhoTldFMWFibTV0YmpkNVJrcGpSRkI1S3pGQ1Rub3JZV3ByTldKclQyNW5LMk00SzJoTU5VcGpWRGxLYjB0aFFWaFhlSE4xY2pkREswMHhWRE0xTlROMk9XSlNXWFJVUVVSNlNXeGlMeXN3TkhOcGJTdEJZWEJ3T0U5blZsUmFOVkpaYzFSdGJUbDZhREJ2ZW01a2VtZHZaelJYTjNWdlluaDNVVmx2TTFkMlJFUTVjMHBYTldGU1JFVTVkVXcyVUV0bk5EUXdNRGxxWjIxdVIyWkpOblZFUVRod01WWlVNRzk0WmpKb2EydEVMMnN4VFVwTmJXUk9ORlJTTlZwclVIaFJTRkF3VkZGa2JVSllaSFlyUzBFd09EVk9jMm95UVdGTVNqaHBVR0psTjFKUVpIRnNVM2hEYzFvMFVreG9UREF4U25BM00wbHBWM2hRYzJoM0syOWtOM2RMUkdOVFpuQXZlSFF4VkZWS1RETXdiRzB4YlRGMWN6SXhUbnBN

In [7]:
aws.clean_up()

cleaning up aws resources
cleaning up s3


AttributeError: 'AWSManager' object has no attribute 'bucket_name'

In [ ]:
import boto3
aws_lambda = boto3.client('lambda', region_name='ap-northeast-1')
with open("tmp.zip", 'rb') as f:
    response = aws_lambda.create_function(
        Code={
            'ZipFile': f.read()
        },
        Description='BusSim handler',
        FunctionName="test",
        Handler='lambda_function.lambda_handler',
        MemorySize=512,
        Publish=True,
        Role="arn:aws:iam::347664766527:role/s3rwRole",
        Runtime='python3.8',
        Timeout=900,
        TracingConfig={
            'Mode': 'Active',
        },
    )

FileNotFoundError: [Errno 2] No such file or directory: 'tmp.zip'